In [177]:
import pandas as pd
import json
import datetime

In [175]:
file_path = "data/weather_trani/weather_data_consum.json"   # for the consumption data (wider time range)
# file_path = "data/weather_trani/weather_data.json"

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [178]:
data_weather = pd.DataFrame()
start_date = datetime.datetime(2018, 8, 1)


for el in data:
    data_temp = el['main']
    timestamp = start_date
    start_date += timedelta(days=1)
    temp = data_temp['temp']-273
    press = data_temp['pressure']
    humid = data_temp['humidity']
    new_row = {'Timestamp' : timestamp,'temp':temp,'press':press,'humid':humid}

    data_weather = data_weather.append(new_row, ignore_index=True)

C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat

In [179]:
data_weather

,Timestamp,temp,press,humid
0,2018-08-01,21.98,1006,55
1,2018-08-02,21.98,1006,55
2,2018-08-03,21.98,1006,55
3,2018-08-04,21.98,1006,55
4,2018-08-05,21.98,1006,55
...,...,...,...,...
1192,2021-11-05,21.98,1006,55
1193,2021-11-06,21.98,1006,55
1194,2021-11-07,21.98,1006,55
1195,2021-11-08,21.98,1006,55


Get the right Timestamp and compute the mean power value of consumption and production.

In [180]:
from datetime import datetime 

In [186]:
building_consumption = pd.read_csv("data/Office building.csv")
building_production = pd.read_csv("data/PV plant.csv")

print(f"building_consumption COLUMNS: {building_consumption.columns}")
print(f"building_production COLUMNS: {building_production.columns}")

data_series = []
time_series = []

for index, row in building_consumption.iterrows():
    timestamp_string = row['Timestamp']
    # Split the string into time and timezone parts
    time_part, timezone_part = timestamp_string.split("+")
    timestamp_string = datetime.strptime(time_part, '%H:%M:%S')

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))

    dt = datetime(row['Year'], row['Month'], row['Day'])
    

    data_series.append(dt)
    time_series.append(timestamp_string)
building_consumption['Timestamp'] = data_series
building_consumption['Timestamp_clock'] = time_series


#avg_power_consum = building_consumption.groupby('Timestamp')['Power (W)'].mean()
#avg_power_consum = (pd.DataFrame(avg_power_consum))

building_consumption COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)', 'Energy (Wh)'], dtype='object')
building_production COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)'], dtype='object')


In [187]:
building_consumption

,Year,Month,Day,Timestamp,Power (W),Energy (Wh),Timestamp_clock
0,2018,6,12,2018-06-12,59293.4,9882.233333,1900-01-01 14:20:00
1,2018,6,12,2018-06-12,NaN,9882.233333,1900-01-01 14:20:00
2,2018,6,12,2018-06-12,25377.2,4229.533333,1900-01-01 14:30:00
3,2018,6,12,2018-06-12,22612.7,3768.783333,1900-01-01 14:40:00
4,2018,6,12,2018-06-12,22534.2,3755.700000,1900-01-01 14:50:00
...,...,...,...,...,...,...,...
134376,2021,1,2,2021-01-02,54586.3,9097.716667,1900-01-01 00:10:00
134377,2021,1,2,2021-01-02,54545.5,9090.916667,1900-01-01 00:20:00
134378,2021,1,2,2021-01-02,55871.6,9311.933333,1900-01-01 00:30:00
134379,2021,1,2,2021-01-02,55195.0,9199.166667,1900-01-01 00:40:00


In [182]:
data_series = []
time_series = []

for index, row in building_production.iterrows():
    timestamp_string = row['Timestamp']

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))
    
    dt = datetime(row['Year'], row['Month'], row['Day'])

    data_series.append(dt)
    time_series.append(timestamp_string)

building_production['Timestamp'] = data_series
building_production['Timestamp_clock'] = time_series

#avg_power_product = building_production.groupby('Timestamp')['Power (W)'].mean()
#avg_power_product = (pd.DataFrame(avg_power_product))

In [170]:
building_production = building_production.drop(columns=['Year','Month','Day'])
building_production

,Timestamp,Power (W),Timestamp_clock
0,2018-08-01,-0.64352,01:00:00+1
1,2018-08-01,-0.64609,01:10:00+1
2,2018-08-01,-0.64651,01:20:00+1
3,2018-08-01,-0.64493,01:30:00+1
4,2018-08-01,-0.64638,01:40:00+1
...,...,...,...
52413,2019-08-01,33.77210,00:00:00+1
52414,2019-08-01,33.85810,00:10:00+1
52415,2019-08-01,33.86440,00:20:00+1
52416,2019-08-01,33.66650,00:30:00+1


In [183]:
building_consumption = building_consumption.drop(columns=['Year','Month','Day'])
building_consumption

,Timestamp,Power (W),Energy (Wh),Timestamp_clock
0,2018-06-12,59293.4,9882.233333,14:20:00+1
1,2018-06-12,NaN,9882.233333,14:20:00+1
2,2018-06-12,25377.2,4229.533333,14:30:00+1
3,2018-06-12,22612.7,3768.783333,14:40:00+1
4,2018-06-12,22534.2,3755.700000,14:50:00+1
...,...,...,...,...
134376,2021-01-02,54586.3,9097.716667,00:10:00+1
134377,2021-01-02,54545.5,9090.916667,00:20:00+1
134378,2021-01-02,55871.6,9311.933333,00:30:00+1
134379,2021-01-02,55195.0,9199.166667,00:40:00+1


In [129]:
# merging on power mean

avg_power_product['Power (W)'] = avg_power_product['Power (W)'] + avg_power_consum[avg_power_consum.index.isin(avg_power_product.index)]['Power (W)']


In [171]:
# merging on power 

building_production['Power (W)'] = building_production['Power (W)'] + building_consumption[building_consumption.index.isin(building_production.index)]['Power (W)']


In [184]:
df_merged_power_consum = building_consumption.merge(data_weather, on='Timestamp')
df_merged_power_product = building_production.merge(data_weather, on='Timestamp')

In [185]:
df_merged_power_consum.to_csv('data/exported_merged_power_consum.csv')
#df_merged_power_product.to_csv('data/exported_merged_power_product.csv')

Energy (Wh) consumption/production plotting